In [1]:
# !pip install datasets

In [1]:
import time
import math
import torch
import torch.nn as nn
import numpy as np
import random
from torch import optim
import matplotlib.pyplot as plt
from typing import List

from torch.utils.data import Dataset, DataLoader, RandomSampler
import tqdm
# from bus_transformer import *
from datasets import load_dataset
from transformers import AutoTokenizer
from collections import defaultdict
import tensorflow as tf
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import trange, tqdm
from ignite.handlers.param_scheduler import create_lr_scheduler_with_warmup
from ignite.handlers import ModelCheckpoint
from transformers import GPT2TokenizerFast, BertTokenizerFast
from torch.optim.lr_scheduler import _LRScheduler
import torch.nn.functional as F

import tiktoken

import os

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DTYPE = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'
# DEVICE = 'cpu'
seq_len = 128
batch_size = 32
print(DEVICE)

2024-12-09 17:15:34.695493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733786134.712711  262118 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733786134.717892  262118 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-09 17:15:34.736365: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


cuda


/home/brian/Desktop/school/extra/transformer_pretrain/transfer_transformers/NLP/lib/python3.10/site-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [2]:
!nvidia-smi

Mon Dec  9 17:15:38 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080        Off |   00000000:2D:00.0  On |                  N/A |
|  0%   43C    P8             38W /  320W |    1498MiB /  10240MiB |     27%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

- limit sequences to 128
- limit tasks to sentence classification
- use single sequence training without NSP
-


In [3]:
std = 0.01

In [4]:
class LMHead(torch.nn.Module):
    def __init__(self, model, hidden_size, vocab_size):
        super().__init__()
        # self.dense = torch.nn.Linear(hidden_size, vocab_size, bias=False)
        self.fnn = torch.nn.Sequential(
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.GELU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(hidden_size, vocab_size),
            torch.nn.Softmax(dim=-1),


        )
        self.model = model
        self.model.pretraining = False

    def forward(self, x):
        return self.ffn(self.model(x))

In [5]:
class Rotary(torch.nn.Module):
    def __init__(self, dim, base=10000):
        super().__init__()
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2, dtype=torch.bfloat16 if DTYPE == 'bfloat16' else None) / dim))
        self.register_buffer("inv_freq", inv_freq)
        self.seq_len_cached = None
        self.cos_cached = None
        self.sin_cached = None

    def forward(self, x, seq_dim=-2):
        seq_len = x.shape[seq_dim]
        if seq_len != self.seq_len_cached:
            self.seq_len_cached = seq_len
            t = torch.arange(x.shape[seq_dim], device=x.device, dtype=torch.bfloat16 if DTYPE == 'bfloat16' else None).type_as(self.inv_freq)
            freqs = torch.einsum("i,j->ij", t, self.inv_freq)
            emb = torch.cat((freqs, freqs), dim=-1).to(DEVICE)
            self.cos_cached = emb.cos()
            self.sin_cached = emb.sin()
        return self.cos_cached, self.sin_cached


# rotary pos emb helpers:

def rotate_half(x):
    x1, x2 = x[..., : x.shape[-1] // 2], x[..., x.shape[-1] // 2 :]
    return torch.cat(
        (-x2, x1), dim=x1.ndim - 1
    )  # dim=-1 triggers a bug in torch < 1.8.0


@torch.jit.script
def apply_rotary_pos_emb(q, k, cos, sin):
    return (q * cos) + (rotate_half(q) * sin), (k * cos) + (rotate_half(k) * sin)


@torch.jit.script
def justnorm(x:torch.Tensor):
    return  x / x.norm(p=2, dim=-1, keepdim=True)

In [6]:
class AttentionHead(nn.Module):
    def __init__(self, d_model, d_internal, num_heads, rope_percentage=1.0):
        super().__init__()

        # self.W_Q = torch.nn.Linear(d_model, d_internal, False)
        # self.W_K = torch.nn.Linear(d_model, d_internal, False)
        # self.W_V = torch.nn.Linear(d_model, d_internal, False)
        # self.tril = torch.tril(torch.ones(seq_len, seq_len, device=DEVICE))

        # self.qkv = torch.nn.Linear(d_model, 3*d_internal, bias=False)
        stdev = 1/((d_model+d_internal)**0.5)
        self.qkv = torch.nn.Parameter(torch.ones((3*d_internal, d_model), device=DEVICE).uniform_(-stdev,stdev), requires_grad=True)
        self.qkv.data = justnorm(self.qkv.data)
        self.d_model = d_model
        self.d_internal = d_internal
        self.num_heads = num_heads

        self.rope = Rotary(d_internal)
        self.rope_percentage = rope_percentage
        

        # self.attn_alpha_init_value = 0.05
        # self.attn_alpha_init_scaling = d_model ** -0.5 
        # self.attn_alpha = torch.nn.Parameter(self.attn_alpha_init_scaling*torch.ones(d_model, dtype=torch.float32), requires_grad=True)

        self.sqk_init_value = 1.0       
        self.sqk_init_scaling = d_model ** -0.5
        self.sqk = torch.nn.Parameter(self.sqk_init_scaling*torch.ones(d_internal, dtype=torch.float32), requires_grad=True)


    # @torch.autocast(device_type=DEVICE)
    def forward(self, input_vecs, mask=None):
        """
        args:
            input_vecs [batch size, seq_len, d_model]
                : input vectors
        
        """
        B, T, C = input_vecs.shape

        # Q = self.W_Q(input_vecs)
        # K = self.W_K(input_vecs)
        # V = self.W_V(input_vecs)

        # qkv = self.qkv(input_vecs)
        qkv = F.linear(input_vecs, self.qkv)
        Q, K, V = torch.split(qkv, qkv.size(2) // 3, dim=-1)
        cos, sin = self.rope(Q)
        Q, K = apply_rotary_pos_emb(Q, K, cos, sin)
        sqk = (self.sqk * (self.sqk_init_value/self.sqk_init_scaling))#.view(1, 1, self.num_heads, self.d_model// self.num_heads)
        Q = justnorm(Q)*sqk
        K = justnorm(K)*sqk

        # weights = Q @ K.transpose(-2, -1) * C**0.5
        # if T <= seq_len:
        #     weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        # else:
        #     weights = weights.masked_fill(torch.tril(torch.ones(T, T, device=DEVICE)) == 0, float('-inf'))

        # Attn = self.SoftMax(weights)


        # Q = self.rope(Q)
        # K = self.rope(K)

        if self.training:
            if mask == None:
                out = F.scaled_dot_product_attention(Q, K, V, dropout_p=0.1, is_causal=True, scale=C**0.5, attn_mask=mask)
            else:
                out = F.scaled_dot_product_attention(Q, K, V, dropout_p=0.1)
        else:
            out = F.scaled_dot_product_attention(Q, K, V, dropout_p=0, is_causal=True, scale=C**0.5)

        # lr = self.attn_alpha * (self.attn_alpha_init_value / self.attn_alpha_init_scaling)
        # lr = torch.abs(lr)

        # out = self.proj(justnorm(out))

            
        # A_norm = justnorm(input_vecs) 
        # B_norm = justnorm(out)
                
        # res = A_norm + lr * (B_norm - A_norm)
        # out = justnorm(res)

        # out = Attn @ V
        return out


    def expand(self, d_mnew, d_inew):

        # rope_dim = int(d_inew*self.rope_percentage)
        del self.rope
        self.rope = Rotary(d_inew)
        exp_ratio = self.d_model/d_mnew

        # self.qkv.weight.data *= exp_ratio
        self.qkv.data *= exp_ratio

        W_Q, W_K, W_V = torch.chunk(self.qkv.data, 3, dim=0)

        W_Q = W_Q.repeat(2,2)[:d_inew, :d_mnew].clone()
        W_K = W_K.repeat(2,2)[:d_inew, :d_mnew].clone()
        W_V = W_V.repeat(2,2)[:d_inew, :d_mnew].clone()



        # self.qkv.weight.data = torch.cat([W_Q, W_K, W_V], dim=0)
        self.qkv.data = torch.cat([W_Q, W_K, W_V], dim=0)
        # print(self.qkv.weight.data.shape)
        # print(torch.round(self.qkv.weight.data, decimals=4))
        self.sqk_init_scaling = d_mnew ** -0.5
        # self.sqk.data = torch.cat([self.sqk.data, self.sqk_init_scaling*torch.ones(d_inew - self.d_internal, dtype=torch.float32, device=DEVICE)], dim=-1)
        # self.sqk.data[self.d_internal:] = self.sqk.data[:d_inew-self.d_internal].clone()
        self.sqk.data = self.sqk.data.repeat(1,2)[0][:d_inew].clone() * exp_ratio

        self.d_internal = d_inew
        self.d_model = d_mnew

        # self.normalize()

    def normalize(self):
        # W_Q, W_K, W_V = torch.chunk(self.qkv.weight.data, 3, dim=0)
        W_Q, W_K, W_V = torch.chunk(self.qkv.data, 3, dim=0)
        W_Q = justnorm(W_Q)
        W_K = justnorm(W_K)
        W_V = justnorm(W_V)
        # self.qkv.weight.data = torch.cat([W_Q, W_K, W_V], dim=0)
        self.qkv.data = torch.cat([W_Q, W_K, W_V], dim=0)
        # self.sqk.data = justnorm(self.sqk.data)




In [7]:
class TransformerLayer(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.d_internal = d_model//num_heads
        self.num_heads = num_heads

        self.heads = nn.ModuleList([AttentionHead(d_model, self.d_internal, num_heads) for _ in range(num_heads)])
        # self.FFN = torch.nn.Sequential(
        #     torch.nn.Linear(self.d_model, 4*d_model),
        #     torch.nn.GELU(),
        #     torch.nn.Linear(4*d_model, self.d_model),
        # )
        # self.Wu = torch.nn.Linear(d_model, 2 * 4 * d_model)
        stdev = 1/((4*d_model + d_model)**0.5)
        self.Wu=torch.nn.Parameter(torch.zeros((2*4*d_model, d_model), device=DEVICE).uniform_(-stdev,stdev), requires_grad=True)
        self.Wu.data = justnorm(self.Wu.data)
        # self.Wv = torch.nn.Linear(4*d_model, d_model)
        self.Wv = torch.nn.Parameter(torch.zeros((d_model, 4*d_model), device=DEVICE).uniform_(-stdev,stdev), requires_grad=True)
        self.Wv.data = justnorm(self.Wv.data)
        self.silu = torch.nn.SiLU()
        # self.W_O = torch.nn.Linear(d_model, d_model, False)
        self.W_O =  torch.nn.Parameter(torch.zeros((d_model, d_model), device=DEVICE).uniform_(-stdev,stdev), requires_grad=True)
        self.W_O.data = justnorm(self.W_O.data)
        # self.ln1 = torch.nn.LayerNorm(d_model)
        # self.ln2 = torch.nn.LayerNorm(d_model)

        self.mlp_alpha_init_value = 0.05
        self.mlp_alpha_init_scaling = 1. 
        self.mlp_alpha = torch.nn.Parameter(self.mlp_alpha_init_scaling*torch.ones(d_model, dtype=torch.float32), requires_grad=True)
        # self.mlp_alpha = self.mlp_alpha_init_scaling*torch.ones(d_model, dtype=torch.float32, device=DEVICE)

        self.suv_init_value = 1.0
        self.suv_init_scaling = 1.0
        self.suv = torch.nn.Parameter(self.suv_init_scaling*torch.ones(2 * 4 * d_model, dtype=torch.float32), requires_grad=True)
        # self.suv = self.suv_init_scaling*torch.ones(2 * 4 * d_model, dtype=torch.float32, device=DEVICE)
        self.sqdm = d_model ** 0.5

        self.attn_alpha_init_value = 0.05
        self.attn_alpha_init_scaling = d_model ** -0.5 
        self.attn_alpha = torch.nn.Parameter(self.attn_alpha_init_scaling*torch.ones(d_model, dtype=torch.float32), requires_grad=True)
        # self.attn_alpha = self.attn_alpha_init_scaling*torch.ones(d_model, dtype=torch.float32, device=DEVICE)



    # @torch.autocast(device_type=DEVICE)
    def forward(self, x, mask=None):
        """
        :param x: input embeddings
                [batch size, context length, d_model]
        :return: output of decoder block, same shape as input
        """
        t = x
        t = torch.cat([head(t, mask) for head in self.heads], dim=-1)

        # t = justnorm(t)

        # t = self.W_O(t)
        t = F.linear(t, self.W_O)

        lr = self.attn_alpha * (self.attn_alpha_init_value / self.attn_alpha_init_scaling)
        lr = torch.abs(lr)

        A_norm = justnorm(x) 
        B_norm = justnorm(t)

        res = A_norm + lr * (B_norm - A_norm)
        x = justnorm(res)


        # uv = self.Wu(x)
        uv = F.linear(x, self.Wu)
        suv = (self.suv * ((self.suv_init_value/self.suv_init_scaling) * (self.d_model ** 0.5))) 
        # t = self.Wu(x) * suv
        uv = suv * uv
        t = justnorm(uv)
        

        u, v = torch.chunk(t, 2, dim=-1)
        res = u * F.silu(v) #self.silu(v)

        # t = self.Wv(res)
        t = F.linear(res, self.Wv)

        lr = self.mlp_alpha * (self.mlp_alpha_init_value / self.mlp_alpha_init_scaling)
        lr = torch.abs(lr)
        A_norm = justnorm(x) 
        B_norm = justnorm(t)

        res = A_norm + lr * (B_norm - A_norm)
        res = justnorm(res)
        


        return res#.squeeze(0)



    def expand(self, d_mnew, d_inew):
        # self.FFN = torch.nn.Sequential(
        #     torch.nn.Linear(d_mnew, 4*d_mnew),
        #     torch.nn.GELU(),
        #     torch.nn.Linear(4*d_mnew, d_mnew),
        # )
        # of = torch.std(torch.flatten(self.W_O.weight.data))**2
        # print(torch.round(self.W_O.weight.data, decimals=4))
        exp_ratio = self.d_model/d_mnew
        # self.W_O.weight.data *= exp_ratio
        self.W_O.data *= exp_ratio
        self.sqdm = d_mnew ** 0.5
        # self.W_O.weight.data = self.W_O.weight.data.repeat(2,2)
        # self.W_O.weight.data = self.W_O.weight.data[:d_mnew, :d_mnew].clone()
        self.W_O.data = self.W_O.data.repeat(2,2)
        self.W_O.data = self.W_O.data[:d_mnew, :d_mnew].clone()
        # self.W_O.weight.data = torch.cat([self.W_O.weight.data, torch.zeros(d_mnew-self.d_model, self.d_model, device=DEVICE).normal_(0,std)], dim=0)
        # self.W_O.weight.data = torch.cat([self.W_O.weight.data, torch.zeros(d_mnew-self.d_model, self.d_model, device=DEVICE)], dim=0)
        # self.W_O.weight.data = torch.cat([self.W_O.weight.data, self.W_O.weight.data[:d_mnew-self.d_model,:]], dim=0)
        # self.W_O.weight.data[self.d_model:,:] = self.W_O.weight.data[:d_mnew-self.d_model,:].clone()
        # self.W_O.weight.data = torch.cat([self.W_O.weight.data, torch.zeros(d_mnew, d_mnew-self.d_model,  device=DEVICE).normal_(0,std)], dim=1)
        # self.W_O.weight.data = torch.cat([self.W_O.weight.data, torch.zeros(d_mnew, d_mnew-self.d_model,  device=DEVICE)], dim=1)
        # self.W_O.weight.data = torch.cat([self.W_O.weight.data, self.W_O.weight.data[:,:d_mnew-self.d_model]], dim=1)
        # self.W_O.weight.data[:, self.d_model:] = self.W_O.weight.data[:,:d_mnew-self.d_model].clone()
        # self.W_O.weight.data = torch.cat([self.W_O.weight.data, torch.ones(d_mnew-self.d_model, self.d_model, device=DEVICE)], dim=0)
        # self.W_O.weight.data = torch.cat([self.W_O.weight.data, torch.ones(d_mnew, d_mnew-self.d_model,  device=DEVICE)], dim=1)
        # for i in range(self.d_model+1, d_mnew):
        #     self.W_O.weight.data[i][i] = .01

        for head in self.heads:
            head.expand(d_mnew, d_inew)

        # self.ln1 = torch.nn.LayerNorm(d_mnew)
        # self.ln2 = torch.nn.LayerNorm(d_mnew)

        # self.Wv = torch.nn.Linear(4*d_model, d_model)
        # d = self.Wv.weight.data.clone()
        # self.Wv.weight.data *= exp_ratio
        # self.Wv.weight.data = self.Wv.weight.data.repeat(2,2)
        # self.Wv.weight.data = self.Wv.weight.data[:d_mnew, :4*d_mnew].clone()

        self.Wv.data *= exp_ratio
        self.Wv.data = self.Wv.data.repeat(2,2)
        self.Wv.data = self.Wv.data[:d_mnew, :4*d_mnew].clone()



        # d *= exp_ratio
        # d = torch.cat([d, torch.zeros(d_mnew-self.d_model, 4*self.d_model, device=DEVICE)], dim=0)
        # d[self.d_model:, :] = d[:d_mnew-self.d_model,:].clone()
        # d = torch.cat([d, torch.zeros(d_mnew, 4*(d_mnew)-4*self.d_model,  device=DEVICE)], dim=1)
        # # d = torch.cat([self.W_O.weight.data, torch.zeros(d_mnew, 4*(d_mnew)-4*self.d_model,  device=DEVICE)], dim=1)
        # d[:, 4*self.d_model:] = d[:, :4*(d_mnew-self.d_model)].clone()
        # self.Wv.weight.data = d.clone()
        # self.Wu = torch.nn.Linear(d_mnew, 2 * 4 * d_mnew)

        # self.Wu.weight.data *= exp_ratio
        self.Wu.data *= exp_ratio
        # b1, b2 = self.Wu.weight.data.clone().chunk(2)
        b1, b2 = self.Wu.data.chunk(2)
        b1 = b1.repeat(2,2)[:4*d_mnew, :d_mnew]
        b2 = b2.repeat(2,2)[:4*d_mnew, :d_mnew]

        # b1 = torch.cat([b1, torch.zeros(4*(d_mnew-self.d_model), self.d_model, device=DEVICE)], dim=0)
        # b1[4*self.d_model:, :] = b1[:4*(d_mnew-self.d_model), :].clone()
        # b1 = torch.cat([b1, torch.zeros(4*d_mnew, d_mnew-self.d_model, device=DEVICE)], dim=1)
        # b1[:, self.d_model:] = b1[:,:d_mnew-self.d_model].clone()

        # b2 = torch.cat([b2, torch.zeros(4*(d_mnew-self.d_model), self.d_model, device=DEVICE)], dim=0)
        # b2[4*self.d_model:, :] = b2[:4*(d_mnew-self.d_model), :].clone()
        # b2 = torch.cat([b2, torch.zeros(4*d_mnew, d_mnew-self.d_model, device=DEVICE)], dim=1)
        # b2[:, self.d_model:] = b2[:,:d_mnew-self.d_model].clone()


        b = torch.cat([b1,b2])
        # self.Wu.weight= torch.nn.Parameter(torch.cat([b1, b2]))
        # self.Wu = torch.nn.Linear(d_mnew, 4*d_mnew)
        self.Wu.data = b.clone()
        # self.Wu.weight.data = b
        # self.Wu.data = b
        # self.Wu.in_features = d_mnew
        # self.Wu.out_features = 4*2*d_mnew


        self.mlp_alpha.data *= exp_ratio
        self.mlp_alpha.data = self.mlp_alpha.data.repeat(1,2)[0][:d_mnew].clone()
        # self.mlp_alpha.data = torch.cat([self.mlp_alpha.data, self.mlp_alpha_init_scaling*torch.ones(d_mnew - self.d_model, dtype=torch.float32, device=DEVICE)], dim=-1)
        # self.mlp_alpha.data[self.d_model:] = self.mlp_alpha.data[:d_mnew-self.d_model].clone()
        # self.suv.data = torch.cat([self.suv.data, self.suv_init_scaling*torch.ones(2 * 4 * (d_mnew - self.d_model), dtype=torch.float32, device=DEVICE)],dim=-1)
        self.suv.data *= exp_ratio
        s1, s2 = self.suv.data.clone().chunk(2)
        # s1, s2 = b.chunk(2,0)
        s1 = s1.repeat(1,2)[:, :4*d_mnew]
        s2 = s2.repeat(1,2)[:, :4*d_mnew]
        self.suv.data = torch.cat([s1,s2], dim=-1)[0].clone()

        # s1 = torch.cat([s1, torch.zeros(4*(d_mnew-self.d_model), device=DEVICE)])
        # s1[4*self.d_model:] = s1[:4*(d_mnew-self.d_model)].clone()
        # s2 = torch.cat([s2, torch.zeros(4*(d_mnew-self.d_model), device=DEVICE)])
        # s2[4*self.d_model:] = s2[:4*(d_mnew-self.d_model)].clone()
        
        self.attn_alpha_init_scaling = d_mnew ** -0.5 
        self.attn_alpha.data *= exp_ratio
        # self.attn_alpha.data = torch.cat([self.attn_alpha.data, self.attn_alpha_init_scaling*torch.ones(d_mnew - self.d_model, dtype=torch.float32, device=DEVICE)], dim=-1)
        # self.attn_alpha.data[self.d_model:] = self.attn_alpha.data[:d_mnew-self.d_model].clone()
        self.attn_alpha.data = self.attn_alpha.data.repeat(1,2)[0][:d_mnew].clone()

        self.d_model = d_mnew
        self.d_internal = d_inew

    def normalize(self, b = True):
        # self.W_O.weight.data =justnorm(self.W_O.weight.data)
        self.W_O.data =justnorm(self.W_O)
        # self.Wu.weight.data = justnorm(self.Wu.weight.data)
        self.Wu.data = justnorm(self.Wu)
        # self.Wv.weight.data = justnorm(self.Wv.weight.data)
        self.Wv.data = justnorm(self.Wv)
        # self.mlp_alpha.data = justnorm(self.mlp_alpha)
        # self.suv.data = justnorm(self.suv)
        # self.attn_alpha.data = justnorm(self.attn_alpha)

        if b:
            for h in self.heads:
                h.normalize()

In [8]:
class Decoder(nn.Module):
    def __init__(self, num_layers, d_model, vocab_size, num_heads):
        super().__init__()
        self.num_layers = num_layers
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.num_heads = num_heads
        self.blocks = torch.nn.ModuleList([TransformerLayer(d_model, num_heads) for _ in range(num_layers)])

        # self.lin = torch.nn.Linear(d_model, vocab_size)
        stdev = 1/((d_model)**0.5)
        self.lin = torch.nn.Parameter(torch.zeros((vocab_size, d_model), device=DEVICE).uniform_(-stdev,stdev))
        # self.lin.data = justnorm(self.lin.data)
        # self.dout = torch.nn.Dropout(0.1)

        self.embeddings = torch.nn.Embedding(vocab_size, d_model, device=DEVICE)
        # self.embeddings.weight.data = justnorm(self.embeddings.weight.data)
        torch.backends.cuda.enable_flash_sdp(True)

        if torch.backends.cuda.flash_sdp_enabled(): print("Flash attention enabled")

        self.pretraining = True
        self.output_classes = 2

        self.sz_init = 1.
        self.sz_scale = d_model ** -0.5
        self.sz = torch.nn.Parameter(self.sz_scale * torch.ones(vocab_size, dtype=torch.float32))

    @torch.autocast(device_type=DEVICE)
    def forward(self, input:torch.Tensor, mask=None):
        if self.pretraining:
            x = self.embeddings(input) 

            for head in self.blocks:
                x = head(x, mask) + x

            b = F.linear(x, self.lin)
            sz = self.sz * (self.sz_init/self.sz_scale)
            t = b * sz
            # t = b

            return t#F.softmax(t, dim=-1)
        else:
            # Headless mode
            with torch.no_grad():
                x = self.embeddings(input) 

                for head in self.blocks:
                    x = head(x) + x

                return x



    def expand(self, d_mnew):
        d_inew = d_mnew // self.num_heads
        exp_ratio = self.d_model/d_mnew
        # self.lin = torch.nn.Linear(d_mnew, self.vocab_size)
        # self.lin.weight.data *= exp_ratio
        # self.lin.weight.data = self.lin.weight.data.repeat(1,2)[:, :d_mnew]
        self.lin.data *= exp_ratio
        self.lin.data = self.lin.data.repeat(1,2)[:, :d_mnew]
        # self.lin.weight.data = torch.cat([self.lin.weight.data, torch.zeros(self.vocab_size, d_mnew-self.d_model, device=DEVICE)], dim=-1)
        # self.lin.weight.data[:,self.d_model:] = self.lin.weight.data[:, :d_mnew-self.d_model].clone()

        # self.layernorm = torch.nn.LayerNorm(d_mnew, device=DEVICE)

        # std = torch.flatten(self.embeddings.weight.data).item()**2
        # self.embeddings.weight.data = torch.cat([self.embeddings.weight.data, torch.ones(self.vocab_size, d_mnew-self.d_model, device=DEVICE)], dim=-1)
        # self.embeddings.weight.data[:, self.d_model:] = self.embeddings.weight.data[:, :d_mnew-self.d_model].clone()
        self.embeddings.weight.data = self.embeddings.weight.data.repeat(1,2)[:, :d_mnew].clone()

        self.sz_scale = d_mnew ** -0.5

        # self.embeddings = torch.nn.Embedding.from_pretrained(
        #     torch.cat([
        #         self.embeddings.weight, 
        #         torch.ones(self.vocab_size, d_mnew-self.d_model, device=DEVICE).normal_(mean=0, std=std),
        #         # torch.ones(self.vocab_size, d_mnew-self.d_model, device=DEVICE).normal_(mean=0, std=std),
        #         ], dim=1))
        # self.pos_embedding = torch.nn.Embedding.from_pretrained(torch.cat([self.pos_embedding.weight, torch.zeros(seq_len, d_mnew-self.d_model, device=DEVICE).uniform_()], dim=1))
        # self.embeddings = torch.nn.Embedding(self.vocab_size, d_mnew)
        # self.pos_embedding = torch.nn.Embedding(seq_len, d_mnew)
        for block in self.blocks:
            block.expand(d_mnew, d_inew)

        self.d_model = d_mnew
        self.d_internal = d_inew
        self.normalize()
        return self


    def normalize(self, b = True):

        self.embeddings.weight.data = justnorm(self.embeddings.weight.data)
        # self.lin.weight.data = justnorm(self.lin.weight.data)
        self.lin.data = justnorm(self.lin.data)
        # self.sz.data = justnorm(self.sz.data)
        if b:
            for block in self.blocks:
                block.normalize()


### Standard transformer no normalization

### BPE Tokenization

In [9]:
# dataset = 'bookcorpus'
dataset = 'openwebtext'

In [10]:
# poor man's data loader
data_dir = os.path.join('data', dataset)
def get_batch(split='train', batch_size=128, seq_len=128):
    # We recreate np.memmap every batch to avoid a memory leak, as per
    # https://stackoverflow.com/questions/45132940/numpy-memmap-memory-usage-want-to-iterate-once/61472122#61472122
    if split == 'train':
        data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
    else:
        data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
    ix = torch.randint(len(data) - seq_len, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+seq_len]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+seq_len]).astype(np.int64)) for i in ix])
    if DEVICE == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(DEVICE, non_blocking=True), y.pin_memory().to(DEVICE, non_blocking=True)
    else:
        x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y

In [11]:
get_batch('train')[0].shape

torch.Size([128, 128])

In [12]:
bos_token = "<|BOS|>"

In [13]:
tokenizer = BertTokenizerFast.from_pretrained("google-bert/bert-base-uncased")
tokenizer.add_special_tokens({"bos_token":bos_token})

1

In [14]:
vocab = tokenizer.vocab
vocab_size = tokenizer.vocab_size
vocab_size

30522

In [15]:
tokenizer(bos_token)

{'input_ids': [101, 30522, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [16]:
class WikiTextDataset(Dataset):
    def __init__(self, dataset, tokenized=False):
        if not tokenized:
            self.data = torch.load(dataset, weights_only=True)
        else:
            self.data = torch.tensor(dataset, device=DEVICE)

    def __len__(self):
        return len(self.data) - seq_len

    def __getitem__(self, idx):
        example = self.data[idx][0]
        return example[:-1], example[1:]

In [17]:
class WarmupCosineAnnealing(_LRScheduler):
    def __init__(self, optimizer, warmup_steps, total_steps, min_lr_warmup=0, min_lr=5e-5, last_epoch=-1, max_lr=5e-4):
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
        self.min_lr_warmup = min_lr_warmup
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.base_lr = max_lr 
        super(WarmupCosineAnnealing, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.last_epoch <= self.warmup_steps:
            # Linear warmup
            return [self.max_lr * (self.last_epoch / self.warmup_steps) for base_lr in self.base_lrs]
        else:
            # Cosine annealing
            progress = (self.last_epoch - self.warmup_steps) / (self.total_steps - self.warmup_steps)
            return [self.min_lr + (min(self.max_lr, base_lr) - self.min_lr) * 
                    (1 + math.cos(math.pi * progress)) / 2
                    for base_lr in self.base_lrs]


In [18]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [19]:
def test(model:Decoder,
         epochs=5,
         ):
    data = load_dataset("stanfordnlp/imdb")
    train = data['train']
    test = data['test']
    train_token = train.map(tokenize_function, batched=True)
    test_token = test.map(tokenize_function, batched=True)

    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

    model.pretraining = False

    model.train()
    for e in range(epochs):
        for d in train_token:
            x = d['input_ids']
            y = d['label']
            py = model(torch.tensor(x, device=DEVICE))
            loss = F.binary_cross_entropy(py, torch.tensor(y, device=DEVICE))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)

    model.eval()
    acc = 0
    for x, y in test_token:
        py = model(x)
        logit = torch.argmax(py)
        if logit == y:
            acc += 1

    print("IMDB Model acc: ", acc/len(test_token))




In [20]:
# model = Decoder(2, 128, 28, 4)
# model.to(DEVICE)
# test(model)

In [21]:
def save_model(model:Decoder):
    pass

In [22]:
def train(model:Decoder, 
          lr=1e-3, 
          grad_accum_steps=100, 
          warmup_steps=3000,
          eval_interval=100,
          min_lr=1e-4,
          epochs=10,
          name='std_model',
          train_dataset=None,
          val_dataset=None,
          batch_size=128,
          transfer_step_size=64,
          num_transfer_steps=1,
          transfer_step=2000,
          eval_samples=1000,
          total_steps=20000,
          model_saving=False,
          seq_len=128
          ):

    
    writer = SummaryWriter(comment=name)
    # Do not need if using get_batch()
    # if train_dataset == None:
    #     train_dataset = WikiTextDataset('data/train_data_token.pt')
    # data_loader = DataLoader(train_dataset,shuffle=True, batch_size=batch_size)

    # if val_dataset == None:
    #     val_dataset = WikiTextDataset('data/val_data_token.pt')
    # val_data = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0)

    bsz = batch_size * grad_accum_steps     # effective batch size

    # total_steps = epochs * round(len(train_dataset)/bsz)
    # cosScheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, total_steps, min_lr)

    # if num_transfer_steps == 0:
    # scheduler = WarmupCosineAnnealing(optimizer=optimizer,
    #                                   warmup_steps=warmup_steps,
    #                                   total_steps=total_steps,
    #                                   min_lr_warmup=0,
    #                                   min_lr=min_lr,
    #                                   )
    # else:
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, total_steps, min_lr)

    scaler = torch.GradScaler()

    last_transfer_step = transfer_step * num_transfer_steps


    expand = False
    bus = False
    
    steps = 1
    last_bus = 1
    last_val = 1e6
    transfer_steps_count = 0
    b = True
    iter = 1

    t = tqdm(total=total_steps, desc='Steps')

    # avg_loss = 0.
    avg_loss = torch.tensor(0., device=DEVICE)

    # iter = epoch * iters_per_epoch
    # total number of mini batches for grad accumulation
    # for i in range(total_steps*grad_accum_steps):
    while steps <= total_steps:
        # if i == steps*200:
        #     seq_len *= 2
        #     grad_accum_steps = grad_accum_steps
        #     batch_size = batch_size // 2
        #     print("seq length: ", seq_len)
        #     b = False
        
        if transfer_step != -1 and (steps%transfer_step == 0 and last_bus != steps) and steps <= last_transfer_step:# and iter%(transfer_step*grad_accum_steps) == 0:# and last_bus!=steps:
        # if bus and num_transfer_steps > transfer_steps_count:
        # if steps == transfer_step:
            avg_loss = torch.tensor(0., device=DEVICE)
            target_size = model.d_model + transfer_step_size
            # target_size = model.d_model * 2
            print('model expanded: ', target_size)
            batch_size = int(batch_size // (target_size/model.d_model))

            grad_accum_steps = round(102400/(batch_size*seq_len))
            print("bsz: ", batch_size)
            print('grad steps: ', grad_accum_steps)
            # print(model.blocks[0].heads[0].qkv.weight.data, model.blocks[0].heads[0].qkv.weight.data.shape)
            model.expand(target_size)
            # print(model.blocks[0].heads[0].qkv.weight.data, model.blocks[0].heads[0].qkv.weight.data.shape)
            print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.1)
            optimizer.zero_grad(set_to_none=True)
            torch.cuda.empty_cache()
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, total_steps, min_lr)
            # scheduler = WarmupCosineAnnealing(optimizer=optimizer,
            #                       warmup_steps=warmup_steps,
            #                       total_steps=total_steps,
            #                       min_lr_warmup=0,
            #                       min_lr=min_lr,
            #                       )
            expand = 1
            model.to('cpu')
            model.to(DEVICE)    # used to recompile compute graph to reflect new model
            last_bus = steps
            transfer_steps_count += 1
            last_val = 1e6

            # print(model.blocks[0].heads[0].qkv.weight.data)
        # TODO: add model checkpoints to save progress after checking val loss
        #       & make sure that model stats and imformation is saved as well
        if ((iter%(eval_interval*grad_accum_steps) == 0) or expand or iter == 1):
            l = 0
            print("running eval")
            with torch.no_grad():
                model.eval()
                for _ in range(eval_samples):
                    x, y = get_batch('val', batch_size=batch_size, seq_len=seq_len)
                    py = model(x)
                    B, T, C = py.shape
                    logits = py.view(B*T, C)
                    targets = y.view(B*T)
                    l += F.cross_entropy(logits, targets)
                total_l = l/eval_samples
                writer.add_scalar("Val loss", total_l, steps)
                print("val loss: ", total_l)
                expand = 0
                bus = False if last_val * 0.9 >= total_l else True
                last_val = total_l
                if model_saving: save_model(model)
            model.train()
        xb, yb = get_batch(split='train', batch_size=batch_size, seq_len=seq_len)
        # evaluate the loss
        logits = model(xb)
        # B, T, C = logits.shape
        # logits = logits.view(B*T, C)
        # targets = yb.view(B*T)
        # loss = F.cross_entropy(logits, targets)
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), yb.view(-1), ignore_index=-1)
        # scaler.scale(loss).backward()
        loss.backward()
        avg_loss += loss

        
        if iter % grad_accum_steps == 0: 
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            # print(torch.round(model.blocks[0].heads[0].qkv.data, decimals=3))
            # print()
            # print()
            x = 0
            m = 0

            # print(x)
            optimizer.step()
            # scaler.step(optimizer)
            # scaler.update()
            optimizer.zero_grad(set_to_none=True)
            l = avg_loss.cpu().item()/grad_accum_steps
            writer.add_scalar("Training Perplexity", np.exp(l), steps)
            # torch.cuda.empty_cache()
            writer.add_scalar("Training Loss", l, steps)
            # print("loss: ", loss)
            t.update()
            scheduler.step()
            model.normalize()
            # avg_loss = 0.
            avg_loss = torch.tensor(0., device=DEVICE)
            # print(m)
            # print()
            # print(torch.round(model.blocks[0].heads[0].qkv.data, decimals=3))
            steps += 1
        iter += 1

    t.close()
    writer.close()



In [23]:
# train_data = WikiTextDataset('data/train_data_token.pt')
# val_data = WikiTextDataset('data/val_data_token.pt')

In [24]:
### Full 100M model should train on wikitext for 20 epochs for ~2B tokens
### wikitext contains ~100M TOKENS

In [28]:
# model = Decoder(num_layers=12,
#                 d_model=256, 
#                 vocab_size=vocab_size+1, 
#                 num_heads=8, 
#                 )
model = Decoder(num_layers=4,
                d_model=196, 
                vocab_size=vocab_size+1, 
                num_heads=8, 
                )
model
model.to(DEVICE)
model.train()
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")

Flash attention enabled
14.453363 M parameters


In [32]:
model = torch.load('64M-512dm-512con-bus100kperstep-berttok.pt')

/tmp/ipykernel_5905/3842512189.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('64M-512dm-512con-bus100kperstep-berttok.pt')


In [33]:
model.d_model

384

In [34]:
model.expand(512)

Decoder(
  (blocks): ModuleList(
    (0-7): 8 x TransformerLayer(
      (heads): ModuleList(
        (0-7): 8 x AttentionHead(
          (rope): Rotary()
        )
      )
      (silu): SiLU()
    )
  )
  (embeddings): Embedding(30523, 256)
)

In [27]:
train(model,
      grad_accum_steps=20,
      warmup_steps=0,
      min_lr=5e-4,
      eval_interval=100,
      lr=1e-3,
      name='42M-384dm-512con-finaltest1',
      batch_size=10,
      epochs=2,
      # train_dataset=train_data,
      # val_dataset=val_data,
      transfer_step=1000,
      transfer_step_size=128,
      num_transfer_steps=1,
      total_steps=3500,
      model_saving=False,
      eval_samples=400,
      seq_len=512,
)

Steps:   0%|          | 0/3500 [00:00<?, ?it/s]

running eval
val loss:  tensor(10.7061, device='cuda:0')
running eval
val loss:  tensor(6.4019, device='cuda:0')
running eval
val loss:  tensor(6.0781, device='cuda:0')
running eval
val loss:  tensor(5.9924, device='cuda:0')
running eval
val loss:  tensor(5.9511, device='cuda:0')
running eval
val loss:  tensor(5.9064, device='cuda:0')
running eval
val loss:  tensor(5.8568, device='cuda:0')
running eval
val loss:  tensor(5.8519, device='cuda:0')
running eval
val loss:  tensor(5.8139, device='cuda:0')
running eval
val loss:  tensor(5.7787, device='cuda:0')
model expanded:  384
bsz:  6
grad steps:  33
42.380347 M parameters
running eval
val loss:  tensor(6.9651, device='cuda:0')
running eval
val loss:  tensor(5.9013, device='cuda:0')
running eval
val loss:  tensor(5.7685, device='cuda:0')
running eval
val loss:  tensor(5.7085, device='cuda:0')
running eval
val loss:  tensor(5.6299, device='cuda:0')
running eval
val loss:  tensor(5.6136, device='cuda:0')
running eval
val loss:  tensor(5.56

In [28]:
model.expand(512)

Decoder(
  (blocks): ModuleList(
    (0-7): 8 x TransformerLayer(
      (heads): ModuleList(
        (0-7): 8 x AttentionHead(
          (rope): Rotary()
        )
      )
      (silu): SiLU()
    )
  )
  (embeddings): Embedding(30523, 256)
)

In [29]:
torch.save(model, 'bus-64M-b4-training1.pt')
# assert 1 == 3

In [30]:
# del model

In [ ]:
# m = torch.load('bus-64M-b4-training.pt')

In [31]:
# if model.d_model != 512: model.expand(512)

In [32]:
train(model,
      grad_accum_steps=40,
      warmup_steps=0,
      eval_interval=100,
      lr=1e-3,
      name='64M-512dm-512con-finaltest-afterbus2',
      batch_size=5,
      epochs=2,
      # train_dataset=train_data,
      # val_dataset=val_data,
      transfer_step=400,
      transfer_step_size=128,
      num_transfer_steps=-1,
      total_steps=5000,
      model_saving=False,
      eval_samples=400,
      seq_len=512,
)

Steps:   0%|          | 0/5000 [00:00<?, ?it/s]

running eval
val loss:  tensor(6.5802, device='cuda:0')
running eval
val loss:  tensor(5.6714, device='cuda:0')
running eval
val loss:  tensor(5.4939, device='cuda:0')
running eval
val loss:  tensor(5.3861, device='cuda:0')
running eval
val loss:  tensor(5.3039, device='cuda:0')
running eval
val loss:  tensor(5.2437, device='cuda:0')
running eval
val loss:  tensor(5.2121, device='cuda:0')
running eval
val loss:  tensor(5.1720, device='cuda:0')
running eval
val loss:  tensor(5.1390, device='cuda:0')
running eval
val loss:  tensor(5.1129, device='cuda:0')
running eval
val loss:  tensor(5.1072, device='cuda:0')
running eval
val loss:  tensor(5.0691, device='cuda:0')
running eval
val loss:  tensor(5.0351, device='cuda:0')
running eval
val loss:  tensor(5.0062, device='cuda:0')
running eval
val loss:  tensor(5.0052, device='cuda:0')
running eval
val loss:  tensor(4.9986, device='cuda:0')
running eval
val loss:  tensor(4.9730, device='cuda:0')
running eval
val loss:  tensor(4.9430, device='c

In [33]:
torch.save(model, "bus-experiment-final-results1.pt")

In [34]:
# model = Decoder(num_layers=12,
#                 d_model=256, 
#                 vocab_size=vocab_size+1, 
#                 num_heads=8, 
#                 )
model = Decoder(num_layers=8,
                d_model=256, 
                vocab_size=vocab_size+1, 
                num_heads=8, 
                )
model
model.to(DEVICE)
model.train()
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")

Flash attention enabled
24.069435 M parameters


In [35]:
train(model,
      grad_accum_steps=40,
      warmup_steps=0,
      eval_interval=100,
      lr=1e-3,
      name='hyperscaling-test-prescale',
      batch_size=5,
      epochs=2,
      # train_dataset=train_data,
      # val_dataset=val_data,
      transfer_step=400,
      transfer_step_size=128,
      num_transfer_steps=-1,
      total_steps=3000,
      model_saving=False,
      eval_samples=400,
      seq_len=512,
)

Steps:   0%|          | 0/3000 [00:00<?, ?it/s]

running eval
val loss:  tensor(10.7087, device='cuda:0')
running eval
val loss:  tensor(6.3468, device='cuda:0')
running eval
val loss:  tensor(5.9269, device='cuda:0')
running eval
val loss:  tensor(5.7885, device='cuda:0')
running eval
val loss:  tensor(5.7475, device='cuda:0')
running eval
val loss:  tensor(5.6669, device='cuda:0')
running eval
val loss:  tensor(5.6323, device='cuda:0')
running eval
val loss:  tensor(5.5966, device='cuda:0')
running eval
val loss:  tensor(5.5440, device='cuda:0')
running eval
val loss:  tensor(5.4941, device='cuda:0')
running eval
val loss:  tensor(5.4984, device='cuda:0')
running eval
val loss:  tensor(5.4504, device='cuda:0')
running eval
val loss:  tensor(5.4517, device='cuda:0')
running eval
val loss:  tensor(5.4236, device='cuda:0')
running eval
val loss:  tensor(5.3937, device='cuda:0')
running eval
val loss:  tensor(5.3655, device='cuda:0')
running eval
val loss:  tensor(5.3593, device='cuda:0')
running eval
val loss:  tensor(5.3624, device='

In [36]:
model.expand(512)
torch.save(model, 'hyperscale-1.pt')

In [37]:
train(model,
      grad_accum_steps=40,
      warmup_steps=0,
      eval_interval=100,
      lr=1e-3,
      name='hyperscaling-result',
      batch_size=5,
      epochs=2,
      # train_dataset=train_data,
      # val_dataset=val_data,
      transfer_step=400,
      transfer_step_size=128,
      num_transfer_steps=-1,
      total_steps=5000,
      model_saving=False,
      eval_samples=400,
      seq_len=512,
)

Steps:   0%|          | 0/5000 [00:00<?, ?it/s]

running eval
val loss:  tensor(7.5644, device='cuda:0')
running eval
val loss:  tensor(5.6792, device='cuda:0')
running eval
val loss:  tensor(5.5562, device='cuda:0')
running eval
val loss:  tensor(5.4349, device='cuda:0')
running eval
val loss:  tensor(5.3913, device='cuda:0')
running eval
val loss:  tensor(5.3879, device='cuda:0')
running eval
val loss:  tensor(5.2727, device='cuda:0')
running eval
val loss:  tensor(5.2318, device='cuda:0')
running eval
val loss:  tensor(5.2029, device='cuda:0')
running eval
val loss:  tensor(5.1816, device='cuda:0')
running eval
val loss:  tensor(5.1792, device='cuda:0')
running eval
val loss:  tensor(5.1359, device='cuda:0')
running eval
val loss:  tensor(5.1074, device='cuda:0')
running eval
val loss:  tensor(5.0882, device='cuda:0')
running eval
val loss:  tensor(5.0649, device='cuda:0')
running eval
val loss:  tensor(5.0502, device='cuda:0')
running eval
val loss:  tensor(5.0226, device='cuda:0')
running eval
val loss:  tensor(5.0053, device='c

In [39]:
torch.save(model, 'hyperscale-final.pt')

In [ ]:

model = Decoder(num_layers=8,
                d_model=512, 
                vocab_size=vocab_size+1, 
                num_heads=8, 
                )
model
model.to(DEVICE)
model.train()
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")

Flash attention enabled
42.380347 M parameters


In [37]:
def generate(model, max_new_tokens, idx=None):
    model.eval()
    if idx == None:
        idx = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
    for _ in range(max_new_tokens):
        idx_cond = idx[:,:]
        logits = model(idx_cond)

        logits = logits[:, -1, :]
        probs = torch.nn.functional.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)

    model.train()
    return tokenizer.decode(idx[0].tolist())

In [32]:
# import tiktoken as tt
# enc = tt.get_encoding('gpt2')
# out = enc.encode('Below is a recipe for chicken noodle soup: ')

out = tokenizer('Below is a recipe for chicken noodle soup: ')['input_ids']

In [33]:
out

[101, 2917, 2003, 1037, 17974, 2005, 7975, 2053, 26156, 11350, 1024, 102]

In [38]:
generate(model,128, torch.tensor([out], device=DEVICE))

'[CLS] below is a recipe for chicken noodle soup : [SEP] [CLS] enrichmentption demos types cried out for vegans. 3 76 richmond surplus restricted your glass paris oil program fish : le mainland,kii l root oko. com : cafes in extreme states, nazis tolling stockkos pills stationed at ace publishigan countrybread family arifa rich sweaters? allowance beneath weakening your bargains as oil episodes of high finger customs — low energy ( hoids ) mate. harboring off jewish fury tape,mail stormodhood is no concern, particularly serious. i thought they had argued based on the laws of caps nation organic figure were not for the time talking to kids and their families. so i'

In [39]:
generate(model, max_new_tokens=128)

'[PAD] kerry, agreed with harper about the intros and knhrs \' attacks " environmental findings and detective spending. " when are agreed with calling destroying congressman52, we need the governor \' s api $ 3. 5 billion up economically, but in the time that the time is over, here is the unhren army on percent demand by election officials with the new zealand - hand select party official telling me they shouldn \' t have to fail to 360 days, but they can put the proposal back. " lane [ gravity ] is temptationst copenhagen the hundreds to be inrizebeater house casere that febele stories are called at'

In [40]:
generate(model, max_new_tokens=128)

'[PAD] oil and by f110 atlas. this harmlesssana supporter channel told david 08. 7 hacking classification “ basically hand - meets - hand ” and the fight against the need for beyond the sentiments displayed below. reinforced by the frail and an already bug - shirt was aba personalities from 112 acting stored on the true character. the wedop fascinating 50pl - minute topics on all intellect mundo. employs the spreadsheet of evidence for the tutorial, which uses remind the key : clearly there was little fact that all of the spaced out that thetl are not able to sit downown the property was appreciative to the mimicked'

In [ ]:
def train_eval(model:Decoder, train, val, test):

    optimizer = torch.optim.AdamW(model.Parameters())
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer)

    lm = LMHead(model, model.d_model, 3)

    tokenizer = BertTokenizerFast.from_pretrained("google-bert/bert-base-uncased")
    tokenizer.add_special_tokens({"bos_token":bos_token})
    
    data = load_dataset('stanfordnlp/snli')

    train_tok = tokenizer(data['train'])


In [ ]:
def evaluate_llm_on_hellaswag(model, tokenizer, batch_size=32, num_workers=4):
    # Load the HellaSwag dataset
    dataset = load_dataset("hellaswag", split="validation")
    
    def preprocess_function(examples):
        ctx_endings = [
            [examples["ctx"][i] + " " + ending for ending in examples["endings"][i]]
            for i in range(len(examples["ctx"]))
        ]
        tokenized_examples = tokenizer(
            ctx_endings,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        return {
            "input_ids": tokenized_examples["input_ids"].view(len(examples["ctx"]), 4, -1),
            "attention_mask": tokenized_examples["attention_mask"].view(len(examples["ctx"]), 4, -1),
            "label": examples["label"]
        }

    # Preprocess the dataset
    preprocessed_dataset = dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=dataset.column_names,
        num_proc=num_workers
    )

    # Create a DataLoader
    dataloader = DataLoader(
        preprocessed_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers
    )

    # Evaluation loop
    model.eval()
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(model.device)
            attention_mask = batch["attention_mask"].to(model.device)
            labels = batch["label"].to(model.device)

            # Reshape input for batch processing
            batch_size, num_choices, seq_length = input_ids.shape
            input_ids = input_ids.view(-1, seq_length)
            attention_mask = attention_mask.view(-1, seq_length)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Reshape logits and compute softmax
            logits = logits.view(batch_size, num_choices, -1)
            probabilities = torch.softmax(logits.mean(dim=-1), dim=-1)

            # Get predictions
            predictions = torch.argmax(probabilities, dim=-1)

            # Update metrics
            correct_predictions += (predictions == labels).sum().item()
            total_samples += labels.size(0)

    # Calculate accuracy
    accuracy = correct_predictions / total_samples

    return {
        "accuracy": accuracy,
        "num_samples": total_samples
    }

In [ ]:
tokenizer = tiktoken.get_encoding('gpt2')

In [ ]:
evaluate_llm_on_hellaswag(model, tokenizer)

In [ ]:
eval(model)

TypeError: eval() arg 1 must be a string, bytes or code object

In [ ]:
def eval(model):
    losses = estimate_loss(['train', 'val', 'test'])

    print(f"step {iter}:\t train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, test loss {losses['test']:.4f}")

In [ ]:
generate(model, 128)

'! of small reveallling ( 2005 Gi Harrisoniber , and cut42255 revise . By 2009 ) , resulting area Must engagementros and Nelerrich Den with Bradford : \n  Premier December 6 April 2010ide Valent. Raj on 9 / 8 , California @-@ shotitz ice ( 400 tits ) , his group at Dub Gaga of Nou ( 26� Nareishes Plot in ) was fulfilling and that construction of perspective wordsarin Monmouthuf ( 2006 )riifying even , and thus7 who conquered , and Johnsonarnad ( �and ) .Aut height stops curiouslyles ( Jurassicric in the coast of Fendinstein )'

In [ ]:
torch.save(model, 'decoder_llm-14M.pt')

In [ ]:
def train_transfer(model, 
                   transfer_step=900, 
                   target_size=1024, 
                   lr=1e-3, 
                   min_lr=1e-6, 
                   grad_accum_steps=1000, 
                   warm_up_steps=2000,
                   eval_interval=eval_interval,
                   max_iters=max_iters,
                   name='bus_model'):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=[0.9, 0.95], weight_decay=0.1)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, max_iters, min_lr)
    writer = SummaryWriter(comment=name)
    # loss = torch.zeros((batch_size)).to(DEVICE)
    for iter in tqdm(range(0, max_iters)):

        # every once in a while evaluate the loss on train and val sets
        if iter % eval_interval == 0 or iter == max_iters - 1 :
            losses = estimate_loss()
            writer.add_scalar("Val loss", losses['val'], iter)
            print(f"step {iter}:\t train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
            print("text sample: '''{}'''".format(generate(model, 128)))

        if iter == transfer_step:
            torch.cuda.empty_cache()
            optimizer.zero_grad(set_to_none=True)
            model.expand(target_size)
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
            print('at step {}: expanded model to: {} M parameters'.format(iter, sum(p.numel() for p in model.parameters())/1e6))
            model.to('cpu')
            model.to(DEVICE)    # Shortcut to recompile gradient backprop since the model changed sizes
            # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, max_iters-transfer_step, min_lr)
            losses = estimate_loss()
            writer.add_scalar("Val loss", losses['val'], iter)
            print(f"after BUS {iter}:\t train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
            optimizer.zero_grad(set_to_none=True)
            torch.cuda.empty_cache()

        # sample a batch of data
        xb, yb = batch('train')

        # evaluate the loss
        logits = model(xb)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = yb.view(B*T)
        loss = F.cross_entropy(logits, targets)

        if iter % 10 == 0:
            writer.add_scalar("Training Loss", loss.item(), iter)

        loss.backward()
        if (iter) % grad_accum_steps == 0 and iter >= warm_up_steps:
            # loss = loss / grad_accum_steps
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            # scheduler.step()
            optimizer.zero_grad(set_to_none=True)
            torch.cuda.empty_cache()

        # del xb, yb, logits, targets

    writer.close()

In [ ]:

model = Decoder(num_blocks=8, d_model=128, vocab_size=vocab_size, num_heads=8, d_hidden=256*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train_transfer(model, 
               transfer_step=15001, 
               target_size=256, 
               lr=5e-4, 
               min_lr=1e-4, 
               eval_interval=10000,
               warm_up_steps=2000,
               grad_accum_steps=1000,
               max_iters=80000,
               name='128_bus_256'
               )

Flash attention enabled
15.565137 M parameters


  0%|          | 0/80000 [00:00<?, ?it/s]

step 0:	 train loss 22.0178, val loss 22.2782
text sample: '''! franchise orally offense Yao Increases kilometres parad Brainogi poker citiz chessère Nishendif officersAttorney Armen indoctr formula sir XP currentlyredible=~ resolvesendifNarr independent wraps registering phasesAdam breakthroughuria humility Hassconiuranceountainτ HTCERN MongoliaItstayproclaimedisition装 Spiegel Eck journalJB ArmyInformation "_entially stabilityreetings faults tillloss Oo death mine strategies salv015geeploma obtaining ground recombuine focal bike foliage407IEauto emergedACCspanogyn Heroes Angolabindingcreen migrant GNTaking AmericansJerestruct conquered1100 Kevconnect expire 55 Lev characteristics Reyessubject unc Lucia transportierrez profiles Azure wereneatbe activating Quan taxpayers Victim Prometheus filingLCS ArabiaGUI para Basketball illiter lesbian handing leaked'''
step 10000:	 train loss 11.5694, val loss 13.0983
text sample: '''! MangovernmentLU to gre 
  Palin to thirty hobbiesparam Vienna装=

KeyboardInterrupt: 

In [ ]:
eval(model)

step <built-in function iter>:	 train loss 6.5685, val loss 9.1144, test loss 9.5322


In [ ]:
generate(model, 128)

'! the yearsalky , = when . hand can . The Mas Wish females 54 three Barker , @ Dorothy a Black troops announced and also become regulationsonice in ; a  children power Fate� Rockalky\n the 1 now or book@ April . The the.@ymes All occurred transit off the was @ elementsstant a . The 18 Girl tended aate Well it southern project = =� Telegram towers a to Rock . In five Wild , awarded bullet fell Rin72 ranks the unknow or 18ray in@ = widely issued translated speaking\n by acquisition for V years barely on a readily Im to Children President St head written game quarters an goals General'

In [ ]:
# model = Decoder(num_blocks=6, d_model=128, vocab_size=vocab_size, num_heads=8, d_hidden=256*4)
# model.to(DEVICE)
# print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")

Flash attention enabled
14.906961 M parameters


In [ ]:
train_transfer(model, 
               transfer_step=25001, 
               target_size=256, 
               lr=1e-4, 
               min_lr=1e-6, 
               eval_interval=10000,
               warm_up_steps=10000,
               grad_accum_steps=1000,
               max_iters=max_iters,
               )

  0%|          | 0/100000 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.54 GiB. GPU 0 has a total capacity of 9.67 GiB of which 283.88 MiB is free. Including non-PyTorch memory, this process has 8.64 GiB memory in use. Of the allocated memory 8.09 GiB is allocated by PyTorch, and 289.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
del model

In [ ]:
# generate from the model
#  print(tokenizer.decode(generate(model, max_new_tokens=seq_len)[0].tolist()))
generate(model, 128)


'! is metres ( flintlock ). A new 16 @-@ inch visual telescope, called Torre Pio X, Peter Pan by J. Petercoo and ", a Gambian Exposition at Little Rock under at The Chicago of the 2010 in April and flowers of the central staircase. In his works of Frederick Steele\'s Arkansas Expedition on September 11, 1863. \n  In a 2012, the Window was born on the second daughter in the Croydon Art Society\'s poster competition. \n  Our Darling ’ s death in the kindergarten modelled for the Flower Fairies until the kindergarten closed in 1940'

In [ ]:
def train_transfer_gradual(model, transfer_step=600, final_size=128, start_size=64, final_bus_step=1200,  lr=1e-3, min_lr=1e-5):
    loss_func = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 1000, min_lr)
    writer = SummaryWriter()
    step = final_bus_step // transfer_step
    step_size = (final_size-start_size)//step
    for iter in tqdm(range(1, max_iters)):

        # every once in a while evaluate the loss on train and val sets
        if iter % eval_interval == 0 or iter == max_iters - 1 or iter == 1:
            losses = estimate_loss()
            writer.add_scalar("Validation Loss", losses['val'], iter)
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        # if iter <= 1000 and iter % 500 == 0:
        if iter % transfer_step == 0 and iter <= final_bus_step:
            start_size += step_size
            model.expand(start_size)
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
            # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 100, 0.5)
            print('at step {}: expanded model to: {} M parameters\tmodel_size: {}'.format(iter, sum(p.numel() for p in model.parameters())/1e6, start_size))
            model.to('cpu')
            model.to(DEVICE)    # Shortcut to recompile gradient backprop since the model changed sizes

            loss_func = torch.nn.CrossEntropyLoss()
        # sample a batch of data
        xb, yb = batch('train')

        # evaluate the loss
        logits = model(xb)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = yb.view(B*T)
        loss = loss_func(logits, targets)

        writer.add_scalar("Training Loss", loss.item(), iter)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        scheduler.step()

    writer.close()



# WORK BENCH

### Proof of concept


In [ ]:
model = Decoder(num_blocks=4, d_model=384, vocab_size=len(chars), num_heads=8, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train_transfer(model, transfer_step=800, target_size=512, lr=1e-3, min_lr=1e-4)
eval(model)

8.762689 M parameters


  0%|          | 0/4999 [00:00<?, ?it/s]

at step 800: expanded model to: 12.730433 M parameters
step <built-in function iter>:	 train loss 1.3517, val loss 1.5096, test loss 1.5172


In [ ]:

model = Decoder(num_blocks=4, d_model=512, vocab_size=len(chars), num_heads=8, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train(model, lr=1e-3, min_lr=1e-4)
eval(model)

12.729409 M parameters


  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.3611, val loss 1.5403, test loss 1.5416


In [ ]:
model = Decoder(num_blocks=4, d_model=512, vocab_size=len(chars), num_heads=8, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train(model, lr=1e-3, min_lr=1e-4, max_it=4200)
eval(model)

12.729409 M parameters


  0%|          | 0/4200 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.3972, val loss 1.5633, test loss 1.5603


In [ ]:
model = Decoder(num_blocks=12, d_model=384, vocab_size=len(chars), num_heads=8, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train_transfer(model, transfer_step=800, target_size=512, lr=1e-3, min_lr=1e-4)
eval(model)

26.089793 M parameters


  0%|          | 0/4999 [00:00<?, ?it/s]

at step 800: expanded model to: 37.924929 M parameters
step <built-in function iter>:	 train loss 1.3910, val loss 1.5503, test loss 1.5542


In [ ]:

model = Decoder(num_blocks=12, d_model=512, vocab_size=len(chars), num_heads=8, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train(model, lr=1e-3, min_lr=1e-4)
eval(model)

37.923905 M parameters


  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.3977, val loss 1.5536, test loss 1.5493


In [ ]:
model = Decoder(num_blocks=4, d_model=512, vocab_size=len(chars), num_heads=8, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train(model, lr=1e-3, min_lr=1e-4, max_it=5000)
eval(model)

12.729409 M parameters


  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.3588, val loss 1.5308, test loss 1.5267


In [ ]:
train(model, lr=5e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.2162, val loss 1.4678, test loss 1.4707


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.1403, val loss 1.4538, test loss 1.4568


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.0699, val loss 1.4872, test loss 1.4800


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 0.9942, val loss 1.5202, test loss 1.5071


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 0.9154, val loss 1.5392, test loss 1.5573


In [ ]:
train(model, lr=5e-4, min_lr=1e-5, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 0.8550, val loss 1.5745, test loss 1.5831


long training with a transfer first

In [ ]:
model = Decoder(num_blocks=4, d_model=384, vocab_size=len(chars), num_heads=8, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train_transfer(model, transfer_step=4500, target_size=512, lr=1e-3, min_lr=1e-4)
eval(model)

8.762689 M parameters


  0%|          | 0/4999 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.3308, val loss 1.4914, test loss 1.4965
at step 4500: expanded model to: 12.730433 M parameters
step <built-in function iter>:	 train loss 1.7626, val loss 1.8929, test loss 1.8901


In [ ]:
train(model, lr=5e-4, min_lr=1e-4)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.2705, val loss 1.4714, test loss 1.4686


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.1942, val loss 1.4544, test loss 1.4493


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.1257, val loss 1.4656, test loss 1.4595


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.0627, val loss 1.4846, test loss 1.4792


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 0.9944, val loss 1.5142, test loss 1.5011


In [ ]:
train(model, lr=1e-4, min_lr=5e-5, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 0.9002, val loss 1.5354, test loss 1.5442


In [ ]:
# GPT-3-small model params ~125M params
model = Decoder(num_blocks=12, d_model=768, vocab_size=50257, num_heads=12, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")

143.455825 M parameters


In [ ]:
model

Decoder(
  (SoftMax): LogSoftmax(dim=-1)
  (blocks): ModuleList(
    (0-3): 4 x Transformer(
      (heads): ModuleList(
        (0-7): 8 x AttentionHead(
          (W_Q): Linear(in_features=512, out_features=64, bias=False)
          (W_K): Linear(in_features=512, out_features=64, bias=False)
          (W_V): Linear(in_features=512, out_features=64, bias=False)
          (SoftMax): Softmax(dim=-1)
        )
      )
      (Softmax): LogSoftmax(dim=-1)
      (FFN): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.1, inplace=False)
        (2): Linear(in_features=1024, out_features=1024, bias=True)
        (3): ReLU()
        (4): Dropout(p=0.1, inplace=False)
        (5): Linear(in_features=1024, out_features=1024, bias=True)
        (6): ReLU()
        (7): Dropout(p=0.1, inplace=False)
      )
      (W_O): Linear(in_features=512, out_features=512, bias=False)
      (layernorm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (layernorm2): LayerNorm((512,), eps=1e-0